###### 反向传播算法推导(三层神经网络为例)
## 符号说明
- i、j和k分别代表输入层、隐含层和输出层的神经元序号
- $net_x$表示神经元x未经过激活函数的输出
- $y_x$表示神经元x经过激活函数后的输出
- $f(\cdot)$表示激活函数
- $t_k$表示输出层的真实值
- 损失函数$L(\mathcal{W}) = \frac{1}{2}\sum_k (y_k - t_k)^2$(加个系数方便后面求导)

参考下面这个示意图($net_i$和$y_i$都和输入相等)
![](network.jpg)

## 更新隐含层到输出层的权重
针对$W_{j,k}$，计算梯度
    $$
    \frac{\partial{L}}{\partial{W_{j,k}}}=\frac{\partial{L}}{\partial{net_k}}\cdot \frac{\partial{net_k}}{\partial{W_{j,k}}}
    $$

$$\frac{\partial{L}}{\partial{net_k}} = \frac{\partial{L}}{\partial{y_k}} \cdot \frac{\partial{y_k}}{\partial{net_k}}$$

容易得到$$\frac{\partial{L}}{\partial{y_k}} = y_k - t_k$$

$$\begin{eqnarray}
\frac{\partial{L}}{\partial{net_k}} &=& (y_k - t_k) \frac{\partial{y_k}}{\partial{net_k}}\\
&=&(y_k - t_k)f^{'}(net_k)
\end{eqnarray}$$

因为有$$net_k = \sum_x W_{x, k}\cdot y_x$$
所以$$\frac{\partial{net_k}}{\partial{W_{j,k}}} = y_j$$

综上可以得到
$$\frac{\partial{L}}{\partial{W_{j,k}}} = (y_k - t_k)f^{'}(net_k)y_j$$

那么k号输出神经元隐含层到输出层权重参数的梯度为
$$\Delta{W_k} = (y_k - t_k)f^{'}(net_k)Y_{hidden}$$

**注意** 这里约定$W_k$和$Y_{hidden}$都是行向量。

这里有个问题，如果有多个输出怎么办？每个输出都会更新上一层的权重，难道取平均？不对，每个输出神经元对应的都是自己的权重，更新的也只是自己的。
那么隐含层到输出层所有的权重的梯度为(这里没办法编程矩阵运算，先放着吧)

## 更新输入层到隐含层的权重
要求的是损失函数相对于i号神经元(输入层)但j号神经元(隐含层)的权重
$$\frac{\partial{L}}{\partial{W_{i,j}}}=\frac{\partial{L}}{\partial{net_j}}\cdot \frac{\partial{net_j}}{\partial{W_{i,j}}}$$

同隐含层的关系有
$$\frac{\partial{net_j}}{\partial{W_{i,j}}} = y_i$$

$$\frac{\partial{L}}{\partial{net_j}} = \frac{\partial{L}}{\partial{y_j}}\cdot \frac{\partial{y_j}}{\partial{net_j}}$$

这里求导时注意把损失函数展开，因为一个隐含层的神经元对应着多个输出层的神经元。
$$\begin{eqnarray}
\frac{\partial{L}}{\partial{y_j}} &=& \frac{\partial}{\partial{y_j}}\left[ \frac{1}{2}\sum_k (y_k - t_k)^2\right]\\
&=&\sum_k (y_k - t_k)\frac{\partial{y_k}}{\partial{y_j}}\\
&=&\sum_k (y_k - t_k)f^{'}(net_k)W_{j,k}\\
\end{eqnarray}$$

综上，可以得到$$\frac{\partial{L}}{\partial{W_{i,j}}} = f^{'}(net_j) y_i \sum_k (y_k - t_k)f^{'}(net_k)W_{j,k}$$

那么j号神经元参数的梯度为$$\Delta W_j = Y_{input}f^{'}(net_j) \sum_k (y_k - t_k)f^{'}(net_k)(W_{j,k})$$

## 简化计算
找出每一层公共计算的部分，尽量使用矩阵操作，以方便实现。

针对某个参数$W_{x, y}$，损失函数对它的梯度总是可以写成两部分
$$\begin{eqnarray}
\frac{\partial{L}}{\partial{W_{x, y}}} &=& \frac{\partial{L}}{\partial{net_y}} \cdot \frac{\partial{net_y}}{\partial{W_{x,y }}}\\
&=&\frac{\partial{L}}{\partial{net_y}}y_x\\
&\rightarrow& \delta_y \cdot y_x
\end{eqnarray}$$

对$\delta_y$进一步拆分，z是y的下一层神经元
$$\begin{eqnarray}
\delta_y &=& \frac{\partial{L}}{\partial{net_y}}\\
&=& \frac{\partial{L}}{\partial{y_y}}\cdot \frac{\partial{y_y}}{\partial{net_y}}\\
&=& f^{'}(net_y)\frac{\partial{L}}{\partial{y_y}}
\end{eqnarray}$$

而
$$\begin{eqnarray}
\frac{\partial{L}}{\partial{y_y}} &=& \sum_z \frac{\partial{L}}{\partial{net_z}}\cdot \frac{\partial{net_z}}{\partial{y_y}}\\
&=& \sum_z \delta_z \cdot W_{y, z}
\end{eqnarray}$$

那么有
$$\Delta W_{x, y} = \delta_y \cdot y_x$$
而
$$\delta_y = f^{'}(net_y)\sum_z \delta_z\cdot W_{y, z}$$
这就将每层参数的更新形式定下来了，并且相邻层的关系也给出了。

最后一层网络的$\delta$记为$\delta_n$
$$\begin{eqnarray}
\delta_n &=& \frac{\partial{L}}{\partial{net_n}}\\
&=& \frac{\partial{L}}{\partial{y_n}}\cdot \frac{\partial{y_n}}{\partial{net_n}}\\
&=& f^{'}(net_n)\cdot \frac{\partial{L}}{\partial{y_n}}
\end{eqnarray}$$
从$\delta_n$开始可以依次计算其他层的$\delta$。

### 矩阵化
这里的向量指行向量
$$\vec{\delta}_k = \vec{\delta}_{k+1} \times \mathcal{W}^T_{k\rightarrow k+1} \cdot f^{'}(\vec{net}_k)$$

$\mathcal{W}^T_{k\rightarrow k+1}$是第k层到k+1层的weights，行数对应k层神经元的个数，列数对应k+1层神经元的个数。注意这里的“$\times$“表示矩阵乘法，而“$\cdot$”计算两个向量对应元素相乘得到的新向量。

$$\Delta \mathcal{W}_{k-1 \rightarrow k} = \vec{x}_k^T \times \vec{\delta}_k$$